In [ ]:
!pip install bcrypt


In [ ]:
# from finvizfinance.screener.overview import Overview
# import os
# from bs4 import BeautifulSoup
# import bs4

# # Override globally
# bs4.builder.builder_registry.builders.clear()
# from bs4.builder import HTMLParserTreeBuilder
# bs4.TreeBuilder.features('lxml')

# # soup = BeautifulSoup(source_data, features="html.parser")
# filters_dict = {'Average Volume':'Over 1M', 'Market Cap.':'+Mid (over $2bln)'}
# # Then rerun your finvizfinance call

# def get_finviz_stocks():

#     foverview = Overview()
#     foverview.set_filter(filters_dict=filters_dict)
#     # df_overview = foverview.()
#     # df_overview = foverview.screener_view()
#     # print('df_overview ',)
#     # if not os.path.exists('out'): #ensures you have an 'out' folder ready
#     #     os.makedirs('out')
#     # df_overview.to_csv('out/Overview.csv', index=False)
# #     tickers = df_overview['Ticker'].to_list()
# #     return tickers

# list_of_stocks = get_finviz_stocks()
# # print('list_of_stocks ',list_of_stocks)

import requests
from bs4 import BeautifulSoup
# Override globally
from bs4.builder import HTMLParserTreeBuilder
from finviz.screener import Screener

def get_finviz_filtered_stocks():
    # headers = {'User-Agent': 'Mozilla/5.0'}
    # url = 'https://finviz.com/screener.ashx?v=111&f=cap_mid+cap_large+cap_mega,sh_avgvol_o1000&r=1'
    # tickers = []
    # page = requests.get(url, headers=headers)
    
    filters = ['ta_rsi_below_30']  # filter list must match Finviz URL params
    stock_list = Screener(filters=filters, table='Overview')
    data = stock_list.data  # list of dicts
    print('data ', data)
    # soup = BeautifulSoup(page.text, features="lxml")
    # for start in range(1, 101, 20):  # Up to 100 stocks
    #     page = requests.get(url + str(start), headers=headers)
    #     soup = BeautifulSoup(page.text, features="lxml")
    #     tables = soup.find_all("table", class_="table-light")

    #     if len(tables) < 2:
    #         break  # no result table found

    #     for row in tables[1].find_all("tr")[1:]:
    #         cols = row.find_all("td")
    #         if len(cols) > 1:
    #             tickers.append(cols[1].text.strip())

    # return tickers

print(get_finviz_filtered_stocks())

In [ ]:
import ta.momentum
import yfinance as yf
import pandas as pd
import ta as ta
from scipy.signal import argrelextrema
import numpy as np

spy = yf.download("PEP", period="20y", interval="1wk")
rsi_input = spy["Close"].squeeze()

spy["RSI"] = ta.momentum.RSIIndicator(rsi_input, window=14).rsi()

def get_swing_lows(prices, order=5):
    idx = argrelextrema(prices.values, np.less_equal, order=order)[0]
    return prices.iloc[idx]

swing_lows = get_swing_lows(spy['Close'], order=5)

# Identify bullish divergence
bullish_divergences = []

for i in range(1, len(swing_lows)):
    prev_date = swing_lows.index[i - 1]
    curr_date = swing_lows.index[i]

    # Extract scalar values using .values[0]
    prev_price = spy.loc[prev_date, 'Close'].values[0]
    curr_price = spy.loc[curr_date, 'Close'].values[0]

    prev_rsi = spy.loc[prev_date, 'RSI'].values[0]
    curr_rsi = spy.loc[curr_date, 'RSI'].values[0]

    if curr_price < prev_price and curr_rsi > prev_rsi and (curr_rsi < 40 and prev_rsi < 35):
        # Store the full datetime index instead of just the date
        bullish_divergences.append((prev_date, curr_date))
print('bullish divergences ', bullish_divergences)

In [ ]:
import ta.momentum
import yfinance as yf
import pandas as pd
import ta as ta
from scipy.signal import argrelextrema
import numpy as np
from datetime import datetime, timedelta

def get_stock_list():
    """Get a list of major stocks to screen"""
    # List of major stock indices and their symbols
    indices = {
        '^GSPC': 'S&P 500',
        '^DJI': 'Dow Jones',
        '^IXIC': 'NASDAQ',
        '^RUT': 'Russell 2000'
    }
    
    all_stocks = set()
    
    # Get components of each index
    for index_symbol in indices.keys():
        try:
            # Get the index components
            index = yf.Ticker(index_symbol)
            components = index.info.get('components', [])
            if components:
                all_stocks.update(components)
        except Exception as e:
            print(f"Error getting components for {index_symbol}: {e}")
    
    # Add some major stocks manually in case the above fails
    major_stocks = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "META", "NVDA", "JPM", "JNJ", "V", 
                   "WMT", "PG", "MA", "UNH", "HD", "BAC", "XOM", "CVX", "PFE", "ABBV"]
    all_stocks.update(major_stocks)
    
    return list(all_stocks)

# def get_qualified_stocks():
#     """Get list of stocks with market cap > 2bn and volume > 1M"""
#     symbols = get_stock_list()
#     qualified_stocks = []
    
#     print("Screening stocks...")
#     for symbol in symbols:
#         try:
#             ticker = yf.Ticker(symbol)
#             info = ticker.info
            
#             # Check market cap (convert to billions)
#             market_cap = info.get('marketCap', 0) / 1e9
#             # Get average volume
#             hist = ticker.history(period="30d")
#             avg_volume = hist['Volume'].mean()
            
#             if market_cap > 2 and avg_volume > 1000000:
#                 qualified_stocks.append(symbol)
#                 print(f"Qualified: {symbol} (Market Cap: ${market_cap:.2f}B, Avg Volume: {avg_volume:,.0f})")
#         except Exception as e:
#             print(f"Error processing {symbol}: {e}")
#             continue
    
#     return qualified_stocks

def find_bullish_divergences(symbol, lookback_days=7):
    """Find bullish divergences in the past week for a given symbol"""
    # Get hourly data for the past week
    end_date = datetime.now()
    start_date = end_date - timedelta(days=lookback_days)
    
    df = yf.download(symbol, start=start_date, end=end_date, interval="1h")
    if df.empty:
        return []
    
    # Calculate RSI
    rsi_input = df["Close"].squeeze()
    df["RSI"] = ta.momentum.RSIIndicator(rsi_input, window=14).rsi()
    
    # Find swing lows
    swing_lows = get_swing_lows(df['Close'], order=5)
    
    # Identify bullish divergence
    bullish_divergences = []
    
    for i in range(1, len(swing_lows)):
        prev_date = swing_lows.index[i - 1]
        curr_date = swing_lows.index[i]
        
        prev_price = df.loc[prev_date, 'Close']
        curr_price = df.loc[curr_date, 'Close']
        
        prev_rsi = df.loc[prev_date, 'RSI']
        curr_rsi = df.loc[curr_date, 'RSI']
        
        if curr_price < prev_price and curr_rsi > prev_rsi and (curr_rsi < 40 and prev_rsi < 35):
            bullish_divergences.append((prev_date, curr_date))
    
    return bullish_divergences

def get_swing_lows(prices, order=5):
    """Find local minima in price series"""
    idx = argrelextrema(prices.values, np.less_equal, order=order)[0]
    return prices.iloc[idx]

# Main execution
# print("Getting qualified stocks...")
# qualified_stocks = get_qualified_stocks()
qualified_stocks = get_stock_list()
print(f"\nFound {len(qualified_stocks)} qualified stocks")

print("\nSearching for bullish divergences...")
stocks_with_divergence = []

for symbol in qualified_stocks:
    try:
        divergences = find_bullish_divergences(symbol)
        if divergences:
            stocks_with_divergence.append({
                'symbol': symbol,
                'divergences': divergences,
                'latest_divergence': divergences[-1][1]  # Store the most recent divergence date
            })
            print(f"Found divergence in {symbol}")
    except Exception as e:
        print(f"Error processing {symbol}: {e}")
        continue

# Sort by most recent divergence
stocks_with_divergence.sort(key=lambda x: x['latest_divergence'], reverse=True)

# Print results
print("\nStocks with bullish divergences in the past week:")
for stock in stocks_with_divergence:
    print(f"\n{stock['symbol']}:")
    for prev_date, curr_date in stock['divergences']:
        print(f"  Divergence: {prev_date} to {curr_date}")

In [ ]:
import ta.momentum
import yfinance as yf
import pandas as pd
import ta as ta
from scipy.signal import argrelextrema
import numpy as np
from datetime import datetime, timedelta

def get_sp500_symbols():
    """Get S&P 500 symbols from Wikipedia"""
    try:
        url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
        tables = pd.read_html(url)
        df = tables[0]
        return df['Symbol'].tolist()
    except Exception as e:
        print(f"Error getting S&P 500 symbols: {e}")
        return ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "META", "NVDA", "JPM", "JNJ", "V"]

def get_stock_data(symbol, days=30):
    """Get historical data for a stock"""
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days)
    return yf.download(symbol, start=start_date, end=end_date, interval="1d")

def calculate_rsi(data):
    """Calculate RSI for a given price series"""
    if data.empty or len(data) < 14:
        return None
    # Convert to 1-dimensional array and create a new Series
    close_prices = close_prices = data["Close"].to_numpy().flatten()  # Convert to 1D numpy array
    return ta.momentum.RSIIndicator(close=pd.Series(close_prices), window=14).rsi()

def get_swing_lows(prices, order=5):
    """Find local minima in price series"""
    idx = argrelextrema(prices.values, np.less_equal, order=order)[0]
    return prices.iloc[idx]

def find_bullish_divergences(data):
    """Find bullish divergences in the price data"""
    if data.empty:
        return []
        
    # Calculate RSI
    data["RSI"] = calculate_rsi(data)
    if data["RSI"].empty:
        return []
    
    # Find swing lows
    swing_lows = get_swing_lows(data['Close'], order=5)
    if len(swing_lows) < 2:
        return []
    
    # Identify bullish divergence
    bullish_divergences = []
    
    for i in range(1, len(swing_lows)):
        prev_date = swing_lows.index[i - 1]
        curr_date = swing_lows.index[i]
        
        # Convert to scalar values
        prev_price = float(data.loc[prev_date, 'Close'])
        curr_price = float(data.loc[curr_date, 'Close'])
        
        prev_rsi = float(data.loc[prev_date, 'RSI'])
        curr_rsi = float(data.loc[curr_date, 'RSI'])
        
        if curr_price < prev_price and curr_rsi > prev_rsi and (curr_rsi < 38 and prev_rsi < 33):
            bullish_divergences.append((prev_date, curr_date))
    
    return bullish_divergences

def main():
    print("Getting S&P 500 symbols...")
    sp500_symbols = get_sp500_symbols()
    print(f"Found {len(sp500_symbols)} S&P 500 stocks")
    
    # Step 1: Filter stocks with RSI < 33
    print("\nFiltering stocks with RSI < 33...")
    low_rsi_stocks = []
    
    for symbol in sp500_symbols:
        try:
            data = get_stock_data(symbol)
            rsi = calculate_rsi(data)
            
            if rsi is not None and not rsi.empty and rsi.iloc[-1] < 33:
                low_rsi_stocks.append(symbol)
                print(f"Found {symbol} with RSI: {rsi.iloc[-1]:.2f}")
        except Exception as e:
            print(f"Error processing {symbol}: {e}")
            continue
    
    print(f"\nFound {len(low_rsi_stocks)} stocks with RSI < 33")
    
    # Step 2: Check for bullish divergences in filtered stocks
    print("\nSearching for bullish divergences...")
    stocks_with_divergence = []
    
    for symbol in low_rsi_stocks:
        try:
            data = get_stock_data(symbol, days=50)  # Get more data for divergence analysis
            divergences = find_bullish_divergences(data)
            
            if divergences:
                stocks_with_divergence.append({
                    'symbol': symbol,
                    'divergences': divergences,
                    'latest_divergence': divergences[-1][1]
                })
                print(f"Found divergence in {symbol}")
        except Exception as e:
            print(f"Error processing {symbol}: {e}")
            continue
    
    # Sort by most recent divergence
    stocks_with_divergence.sort(key=lambda x: x['latest_divergence'], reverse=True)
    
    # Print results
    print("\nStocks with bullish divergences:")
    for stock in stocks_with_divergence:
        print(f"\n{stock['symbol']}:")
        for prev_date, curr_date in stock['divergences']:
            print(f"  Divergence: {prev_date.date()} to {curr_date.date()}")
    
    print(f"\nSummary:")
    print(f"Total S&P 500 stocks: {len(sp500_symbols)}")
    print(f"Stocks with RSI < 33: {len(low_rsi_stocks)}")
    print(f"Stocks with bullish divergences: {len(stocks_with_divergence)}")

if __name__ == "__main__":
    main()

In [6]:
import bcrypt

password = b"testpassword"  # must be in bytes
salt = bcrypt.gensalt(rounds=12)  # 12 is the cost factor
hashed = bcrypt.hashpw(password, salt)

print(hashed.decode())

ModuleNotFoundError: No module named 'bcrypt'